In [ ]:
import yaml
import oemof.solph

from mtress import Location, MetaModel

In [ ]:
cfg_string = """
locations:
  LocationA:
    energy_cost:
      electricity:
        demand_rate: 15000  # €/MW
        working: 300  # €/MWh
      gas:
        working_price: 95  # €/MWh
        demand_rate: 5.5  # €/MW
      wood_pellet: 300  # €/MWh
    
    meteorology:
      temp_air: meteo.csv:temp_air
      temp_soil: meteo.csv:temp_soil
    
    carriers:
      Heat: # class name
        reference_temperature: 10 # °C
        temperature_levels:
          - 35. # °C
          - 60. # °C
          - 80. # °C
      Electricity:
        grid_connection: True

    demands:
      Electricity:
        time_series: demand.csv:electricity
      SpaceHeating:
        time_series: demand.csv:space_heating
    
    components:
      air_source:
        technology: AirHeatExchanger
        parameters:
          nominal_power: 7.
          air_temperatures: .1
      hp0:
        technology: HeatPump
        parameters:
          cop_0_35: 2.
          thermal_power_limit: 0.1 # MW
      hs0:
        technology: HeatStorage
        parameters:
          diameter: 1
          volume: 10
          insulation_thickness: .2
          ambient_temperature: 10.

"""

In [ ]:
cfg = yaml.safe_load(cfg_string)
print(cfg)

In [ ]:
meta_model = MetaModel(config=cfg)

In [ ]:
from mtress._helpers._visualization import generate_graph

In [ ]:
generate_graph(meta_model.energy_system)

In [ ]:
meta_model.energy_system.nodes

In [ ]:
import numpy as np

In [ ]:
v = [ 1.1, 2.2, 3.3, 4.4]

In [ ]:
i = np.searchsorted(v, 1)
v[i-1], i